<a href="https://colab.research.google.com/github/ChristianKitte/SparkProjekt/blob/main/notebook/Wordcount_mit_Spark_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung des Notebooks
Vor der eigentlichen Umsetzung der Aufgabe muss das Notebook vorbereitet werden. ***Python*** und die ***wichtigsten Bibliotheken sind bereits Bestandteil*** der
von Colab bereit gestellten Installation. 

Da Spark mit Java programmiert wurde, wird als erstes ***Java installiert***. In dieser Anwendung wird das Open JDK in der Version 8 verwendet. Die Installation
erfolgt direkt auf der Linux Ebene.

Nachdem Java installiert ist, kann Spark selbst ***heruntergeladen sowie alle Systemvariablen gesetzt werden***. Auch hier wird mit Linuxbefehlen operiert. In dieser Anwendung wird die aktuell neueste Version 3.2 verwendet.

In [ ]:
# Installation  von Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

print("Java ist installiert...")

# Download und Entpacken von Spark (Versionsnummer anpassen!)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

print("Spark ist verfügbar...")

# Setzen der Systemvariablen für Java und Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

print("Umgebungsvariablen sind gesetzt...")

Java ist installiert...
Spark ist verfügbar...
Umgebungsvariablen sind gesetzt...


In [ ]:
# Installation von findspark und pyspark

!pip install findspark
print("FindSpark wurde installiert...")

!pip install pyspark
print("PySpark wurde installiert...")

FindSpark wurde installiert...
PySpark wurde installiert...


In [ ]:
# Initialisieren von findspark

try: 
  import findspark
  from pyspark import SparkContext, SparkConf
  
  findspark.init()
  
  print("FindSpark und PySpark wurden initialisiert")
except ImportError: 
  raise ImportError("Fehler bei der Initialiserung von FindSpark und PySpark")

FindSpark und PySpark wurden initialisiert


# Einlesen und Vorbereiten der Textdatei

Im ersten Abschnitt werden zunächst zwei Methoden definiert.

* Der ersten Methode ***get_file_from_url*** werden als Parameter eine ***URL*** sowie ein ***Speicherort*** angegeben. Bei Ihrem Aufruf lädt die Methode eine Datei von der angegebenen URL herunter und speichert sie in Google Drive ab.

* Die zweite Methode ***cut_file*** nimmt als Parameter einen ***numerischen Start- und Endwert*** sowie die Angabe einer ***Quell- und Zieldatei*** entgegen. Bei Ihrem Aufruf entfernt die Methode alle Zeilen vor bzw. nach den durch Start- und Endwert definierten Zeilenbereich aus der Quelldatei und speichert das Ergebnis in die Zieldatei.

In dem folgenden Block wird dann im Anschluss die Datei mit den gesammelten Werken von Shakespeare von der Seite des MIT ***heruntergeladen*** sowie ***von nicht benötigten Zeilen bereinigt*** und in einer ***neuen Datei*** gespeichert.

In [ ]:
# Erstellen einer Methode, um Dateien aus dem Internet zu laden und zu speichern

import requests 

def get_file_from_url(file_url, place_to_save):
  try:
    req = requests.get(file_url, stream = True) 

    with open(place_to_save, "wb") as file: 
	    for block in req.iter_content(chunk_size = 1024): 
		    if block: 
			    file.write(block) 
     
    print("Die Datei wurde herunter geladen und angelegt: {}".format(file_url))
  
  except ValueError:
    print("Fehler {}".format(ValueError))   

print("Die Funktion get_file_from_url wurde angelegt...")


Die Funktion get_file_from_url wurde angelegt...


In [ ]:
# Erstellen einer Methode, um eine Textdatei am Anfang und am Ende um die jeweils
# angegebene Zahl an Reihen zu beschneiden.

def cut_file(anfang, ende, quelldatei, zieldatei):
  try:
    with open(quelldatei, "r") as source:
      lines = source.readlines()
    
    source.close()

    print("")
    print("Start: {}".format(anfang))
    print("Ende: {}".format(ende))
    print("")

    current_count = 0
  
    with open(zieldatei, "w") as target:
      for line in lines:
        if current_count >= anfang and current_count <= ende:
          target.write(line)

        current_count = current_count + 1   
    
    target.close()

    print("Datei wurde beschnitten...")

  except ValueError:
    print("Fehler {}".format(ValueError))

print("Die Funktion cut_file wurde angelegt...")

Die Funktion cut_file wurde angelegt...


In [ ]:
# Datei von der Quelle nach Colab laden

file_url = "https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt"
place_to_save = "/content/shakespeare.txt"

get_file_from_url(file_url, place_to_save)

print("")
print("Datei wurde vorbereitet...")

Die Datei wurde herunter geladen und angelegt: https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt

Datei wurde vorbereitet...


In [ ]:
# Unnötige Zeilen am Ende und am Start entfernen

file_source = "/content/shakespeare.txt"
file_target = "/content/shakespeare_neu.txt"

cut_file(244,124438,file_source, file_target)

print("")
print("Die Arbeitsdatei ist vorbereitet...")


Start: 244
Ende: 124438

Datei wurde beschnitten...

Die Arbeitsdatei ist vorbereitet...


# Auszählen der Wörter

Um mit Spark arbeiten zu können, muss als erstes eine Verbindung zu Spark in Form eines ***SparkContext*** aufgebaut werden. In dem hier verwendeten Code wird ein SparkContext erzeugt, welcher die Bezeichnung ***WordCounter*** erhält. Er soll lokal laufen und hierbei parallel ***alle verfügbaren Kerne*** verwenden. Dieser Block kann in einer Anwendung nur einmal ausgeführt werden.

Anschließend wird die Textdatei eingelesen und gibt ein ***RDD*** in Form einer Liste von String zurück. In diesen Fall entsprechen die Strings den ***Zeilen der Textdatei***. Die Methode ***map*** führt auf jedes Element des zugrunde liegenden RDD - also den Zeilen der Textdatei - die angegebene Funktion aus.

In dem hier vorliegenden Fall findet zunächst eine Reihe von Ersetzungen (***replace***), dann eine Konvertierung in Kleinbuchstaben (***lower***) und am Schluss eine Filterung (***filter***) auf leere Zeilen statt. Als Ergebnis wird ein neues RDD vom Typ String zurückgegeben. Das ursprüngliche RDD wird ***nicht verändert***. Es ist ***immutable***. Die Verwendung einer FluentApi bewirkt eine übersichtliche Strukturierung des Codes.

In der folgenden Codesequenze wird jedes Listenelement des RDD durch ***flatMap*** in seine einzelnen Wörter aufgeteilt. Für jedes Wort wird ein Tupel erzeugt und zurückgegeben. Da es sich um eine flatMap handelt, verfügt das zurück gegebene RDD nur noch über eine sehr lange ***Liste von Tupel***. Die Funktion ***reduceByKey*** merged im Anschluss die einzelnen Tupel. Als Ergebnis erhält man eine ***Liste von Tupel*** mit ***eindeutigen Wörtern und deren Vorkommen***.

Mit der Methode ***sortBy*** wird auf die ***Anzahl der Wortvorkommen sortiert***. Das zurück gegebene RDD ***sorted_counts*** kann im Anschluss ausgegeben werden, nachdem mit ***collect*** alle Werte eingesammelt wurden.

In [ ]:
# Erzeugen eines Spark Kontext 

sc = SparkContext("local[*]","WordCounter")
sc.setLogLevel("ERROR")

print("Der Spark Kontext wurde angelegt...")

Der Spark Kontext wurde angelegt...


In [ ]:
# Auszählen der Wörter

from pyspark import SparkContext 
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext 
from pyspark.sql import DataFrameReader

lines=sc.textFile(file_target) \
  .map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower()) \
  .filter(lambda linex: linex.strip() != "") 

top_out = 30

print("")
print("Ausgabe der ersten {} Zeilen des Textes".format(top_out))
print("")

for line in lines.collect()[0:top_out]:
  print(line) 

words = lines.flatMap(lambda line: line.split(" ")) \
  .map(lambda word: (word, 1)) \
  .reduceByKey(lambda a,b:a+b)

sorted_counts = words.sortBy(lambda wordCounts: wordCounts[1], ascending=False)

top_length = 30

print("")
print("Ausgabe der {} größten Vorkommen".format(top_length))
print("")

i = 0
for word, count in sorted_counts.collect()[0:top_length]:
    print("{} : {} : {} ".format(i, word, count))
    i += 1


Ausgabe der ersten 30 Zeilen des Textes

1609
the sonnets
by william shakespeare
                     1
  from fairest creatures we desire increase 
  that thereby beauty's rose might never die 
  but as the riper should by time decease 
  his tender heir might bear his memory:
  but thou contracted to thine own bright eyes 
  feed'st thy light's flame with self substantial fuel 
  making a famine where abundance lies 
  thy self thy foe  to thy sweet self too cruel:
  thou that art now the world's fresh ornament 
  and only herald to the gaudy spring 
  within thine own bud buriest thy content 
  and tender churl mak'st waste in niggarding:
    pity the world  or else this glutton be 
    to eat the world's due  by the grave and thee 
                     2
  when forty winters shall besiege thy brow 
  and dig deep trenches in thy beauty's field 
  thy youth's proud livery so gazed on now 
  will be a tattered weed of small worth held:  
  then being asked  where all thy beauty lies